In [1]:
import numpy as np, pylab as plt, pandas as pd
import ase, ase.io, itertools as itr, mendeleev
from bvel_config import principleQuantumNumbers, covalentRadii, oxidationStates, atomicNumbers, atomicLabels, \
    D0_coeficients, alpha_coeficients, Rmin_coeficients, R0_coeficients

In [2]:
from numpy import ravel_multi_index as rav

In [3]:
alkali = 'Li'
#lanthanide = 'Gd'

In [4]:
cif = 'LTS_neutron.cif'

In [5]:
cutoffRadii = 6.

In [6]:
def include_atomsFrom_nns_unitCells(scaledPositions):
    nnsCoordinates = [np.array(i) for i in itr.product([-1,0,1],repeat=3)]
    totalAtomsIncludingNNS = np.concatenate([scaledPositions + nnsCoordinate for nnsCoordinate in nnsCoordinates])
    return totalAtomsIncludingNNS

def combine_atomList(atomList):
    combinedList = atomList[0]
    for listItem in atomList:
        combinedList += listItem
    return combinedList

def get_atomsFromSuperCell(atoms):
    nnsCellCoordinates = [np.array(i) for i in itr.product([-1,0,1],repeat=3)]
    neighborAtomsList = []
    for nnsCellCoordinate in nnsCellCoordinates:
        neighborAtomCoordinates = atoms.get_scaled_positions() + nnsCellCoordinate
        neighborAtoms = atoms.copy()
        neighborAtoms.set_scaled_positions(neighborAtomCoordinates)
        neighborAtomsList.append(neighborAtoms)
    superCellAtoms = combine_atomList(neighborAtomsList)
    return superCellAtoms

class Ions:
    
    def __init__(self,atoms):
        self.atoms = get_atomsFromSuperCell(atoms)
        
    def get_ionsInCutoff(self,position,cutoff):
        lowerBoundary = position - cutoff
        upperBoundary = position + cutoff
        inBoundary = np.all(((self.atoms.get_positions() < upperBoundary) & (self.atoms.get_positions() > lowerBoundary)),axis=1)
        return self.atoms[inBoundary]
    
class UnitCell:
    
    def __init__(self,cif):
        self.atoms = ase.io.read(cif)
        self.latticeParameters = np.linalg.norm(self.atoms.cell,axis=1)
        self.latticeVectors = self.atoms.cell
        
    def get_ionOfType(self,ionLabel):
        atomicNumber = atomicNumbers[ionLabel]
        atoms = self.atoms[self.atoms.numbers==atomicNumber]
        ions = Ions(atoms = atoms)
        return ions
    
    def get_multipleIonsOfType(self,ionLabelList):
        atomicNumberList = [atomicNumbers[ionLabel] for ionLabel in ionLabelList]
        indexOfIons = np.any([(self.atoms.numbers == atomicNumber) for atomicNumber in atomicNumberList],axis=0)
        atoms = self.atoms[indexOfIons]
        ions = Ions(atoms = atoms)
        return ions

In [7]:
unitCell = UnitCell(cif)
mainIons = unitCell.get_ionOfType(alkali)
negativeIons = unitCell.get_ionOfType('O')
positiveIons = unitCell.get_multipleIonsOfType(['Si','La','Ta'])

/Users/Jack/code/miniconda3/lib/python3.7/site-packages/ase/io/cif.py:125: UserWarning: Wrong number of tokens: ['?', "#<--'Last", 'name,', 'first', "name'"]
  warnings.warn('Wrong number of tokens: {0}'.format(tokens))


In [23]:
dx,dy,dz = 0.1,0.1,0.1
resolution = np.array([dx,dy,dz])

In [24]:
sizeOfValenceMismatchArray = tuple((unitCell.latticeParameters / resolution).astype(int))
BVEL = np.ones(sizeOfValenceMismatchArray)

In [25]:
sizeOfValenceMismatchArray

(87, 87, 136)

In [26]:
from scipy.special import erfc

In [27]:
D0 = D0_coeficients[alkali]
a = alpha_coeficients[alkali]
Rmin = Rmin_coeficients[alkali]
R0 = R0_coeficients[alkali]

In [28]:
alkali_atomicNumber = atomicNumbers[alkali]
rcovAlk = covalentRadii[alkali_atomicNumber]

In [29]:
numberOfVoxels = len(BVEL.flatten())
repulsive = []
attractive = []
#for voxelFlatIndex in range(10):
for voxelFlatIndex in range(numberOfVoxels):
    voxelTupleIndex = np.unravel_index(voxelFlatIndex,BVEL.shape)
    # I don't understand this
    voxelScaledPosition = np.array(voxelTupleIndex)/np.array(BVEL.shape).astype(float)
    voxelUnscaledPosition = np.dot(unitCell.latticeVectors.T,voxelScaledPosition)
    
    negativeIonsInCutoff = negativeIons.get_ionsInCutoff(position=voxelUnscaledPosition,cutoff=cutoffRadii)
    positiveIonsInCutoff = positiveIons.get_ionsInCutoff(position=voxelUnscaledPosition,cutoff=cutoffRadii)
    # the above needs to be changed to a position

    # ------------- get attractive variables ---------------------    
    
    distanceToNegativeIons = np.linalg.norm(negativeIonsInCutoff.get_positions() \
                                                   - voxelUnscaledPosition, axis=1)
    Ri = distanceToNegativeIons
    cutIndex = Ri<cutoffRadii
    
    Ri = Ri[cutIndex]
    negativeIonsInCutoff.distance = Ri
    #print(len(Ri))
    
    # ------------- calculate attractive term ---------------------
    
    smin = np.exp(a*(R0-Rmin))
    
    Eattractive_i = D0 * ( (1./smin**2) * ( np.exp(a*(R0-Ri)) - smin )**2 - 1 )
    attractiveTerm = np.sum(Eattractive_i)
    attractive.append(attractiveTerm)
    
    
    # ------------- get repulsive variables ---------------------
    
    positiveIonsInCutoff.distance = np.linalg.norm(positiveIonsInCutoff.get_positions() \
                                                   -voxelUnscaledPosition, axis=1)
    Rj = positiveIonsInCutoff.distance
    cutIndex = Rj<cutoffRadii
    #print(len(Rj))
    
    positiveIonsInCutoff.distance = positiveIonsInCutoff.distance[cutIndex]
    
    positiveIonsInCutoff.covalentRadii = np.array([covalentRadii[atomicNumber] \
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]

    positiveIonsInCutoff.principleQuantumNumbers = np.array([principleQuantumNumbers[atomicNumber] \
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]
    
    positiveIonsInCutoff.oxidationStates = np.array([oxidationStates[atomicNumber] \
                                        for atomicNumber in positiveIonsInCutoff.numbers])[cutIndex]
    
    # ------------- calculate repulsive term ---------------------
    
    C = 14.40198 # conversion factor
    d = positiveIonsInCutoff.distance
    q = positiveIonsInCutoff.oxidationStates
    n = positiveIonsInCutoff.principleQuantumNumbers
    r = positiveIonsInCutoff.covalentRadii
    p = 0.74 * (rcovAlk + r)
    
    Erepulsive_j = C*q/(2*np.sqrt(2*n)) * (erfc(d/p) - erfc(cutoffRadii/p))
    repulsiveTerm = np.sum( Erepulsive_j )
    repulsive.append(repulsiveTerm)
    
    # ------------- sum it up ---------------------
    
    BVEL[voxelTupleIndex] = attractiveTerm + repulsiveTerm 
    
    

KeyboardInterrupt: 

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'

In [ ]:
8.99e9 * 1e10 * (1.602e-19)

In [ ]:
plt.rc('font', **{'family' : 'serif', 'size'   : 18})
fig,ax = plt.subplots(1,1,figsize=(7.5,5))
plt.setp(ax.spines.values(), linewidth=2)

ax.plot(repulsive)
ax2 = ax.twinx()
ax2.plot(attractive,color='C1')

In [ ]:
BVEL = BVEL - BVEL.min()

In [ ]:
#output_filename = 'myBVEL.grd'
#output_filename = cif.split('.')[0] + '.grd'
output_filename = cif.split('.')[0] +'_shift'+ '.grd'

with open(output_filename,"w") as savefile:
    savefile.write("Bond Valence Sum Difference\r") # Title
    from ase.geometry import cell_to_cellpar
    cellParams = cell_to_cellpar(unitCell.latticeVectors) # get ABC alpha, beta, gamma
    savefile.write(" ".join([str(k) for k in cellParams])+"\r" )
    savefile.write(" ".join([str(k) for k in BVEL.shape])+"\r" )
    for i in np.nditer(BVEL.flatten()):
        savefile.write("%.6f  "%(i)) # Write each valence difference value

In [78]:
import scipy.ndimage.measurements as mes
import numpy as np

def testPerc(z,volfrac=False,dimensions=False):
    if not np.any(z):
        return False
    lw, num = mes.label(z)
    len(np.unique(lw))-1 # number of clusters
    area = mes.sum(z, lw, index=np.arange(lw.max() + 1))
    areaImg = area[lw] + lw*0.01
    sliced = mes.find_objects(areaImg == areaImg.max())
    cluster_dimensions = np.array([int(sliced[0][i].stop-sliced[0][i].start) for i in range(len(z.shape))])
    if not dimensions and not volfrac:
        doesPercolate = np.any(((cluster_dimensions/np.array(z.shape))).astype(int))
        return doesPercolate
    elif volfrac:
        # volume of the network / volume of the box around it.
        return areaImg.max()/np.prod(cluster_dimensions)
    elif dimensions:
        # network dimensions / dimensions of the cell
        return cluster_dimensions/np.array(z.shape).astype(float)    

def threshold(r):
    p = r.min().round(2); thresh = False
    while p < 2 and not thresh:
        z2 = r < p
        tp = testPerc(z2)
        if tp:
            thresh = p
        else:
            p+=0.01
    return p

In [113]:
threshold(BVEL)

0.8900000000000006

In [61]:
BVEL.min()

0.0

In [67]:
np.any(BVEL == 0.0)

True

In [68]:
testPerc(BVEL==0.00)

True